In [1]:
import os
from dotenv import load_dotenv
import nest_asyncio
nest_asyncio.apply()
load_dotenv()

False

In [2]:
from llama_index.core import SimpleDirectoryReader
# load lora_paper.pdf documents
documents = SimpleDirectoryReader(input_files=["./data/Lora.pdf"]).load_data()

ValueError: File ./data/Lora.pdf does not exist.

In [ ]:
os.getcwd()


'/teamspace/studios/this_studio'

In [ ]:
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
OPENAI_API_KEY